In [1]:
import pandas as pd
import csv
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import preprocessing
from math import floor, ceil
from pylab import rcParams
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, auc
%matplotlib inline
import os
import random as rn
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
PYTHONHASHSEED=0
tf.random.set_seed(1234)
np.random.seed(1234)
rn.seed(1254)

In [2]:
def splitBIRADSScore(score):
    for i in range(score.size):
        if score[i]>0 and score[i]<4:
            score[i]=1
        if score[i]==4:
            score[i]=2
        if score[i]>4:
            score[i]=3

In [3]:
def removeRADSZero(inData):
    BiClassData = np.copy(inData)
    for i in range(BiClassData.shape[0]):
        if(i<BiClassData.shape[0]):
            if(BiClassData[i, 1]==0):
                # print('hi')
                BiClassData=np.delete(BiClassData, i, 0)
    return BiClassData

In [4]:

trainData = pd.read_csv('CroppedPicsTrain.csv', sep=',', header = None).values
#malignant or benign as y
yMoBTrain=trainData[:,0]
#level of classification as y
yRADSTrain=trainData[:,1]
XTrain = trainData[:, 2:trainData.shape[1]]
#split the BI-RADS y data (yROITrain) into four categories (we may want to consider excluding zero samples if we are getting low validity rates)
#0: inconclusive mammogram (difficult to read or interpret) (so might want to exclude these samples)
#1: probably benign (BI-RADS score of 1-3)
#3: suspicious abnormality (should be investigated further but may not be cancer) (BI-RADS Score of 4)
#4: highly suspicious findings (likely to be malignant) (BI-RADS Score of 5-6)
#note: technically 6 is only obtainable with a biopsy but beause these patients were followed up on to determine if it was malignant some have a score of 6
#read about BI-RADS scores here: https://www.cancer.org/cancer/breast-cancer/screening-tests-and-early-detection/mammograms/understanding-your-mammogram-report.html
splitBIRADSScore(yRADSTrain)

#exclude all the data with a BIRAD score of 0
biClassTrain = removeRADSZero(trainData)
XBiClassTrain = biClassTrain[:, 2:biClassTrain.shape[1]]
yBiClassTrain = biClassTrain[:, 0]

testData = pd.read_csv('CroppedPicsTest.csv', sep=',', header = None).values
#malignant or benign as y
yMoBTest=testData[:,0]
#level of classification as y
yRADSTest=testData[:,1]
splitBIRADSScore(yRADSTest)
XTest = testData[:, 2:testData.shape[1]]

biClassTest = removeRADSZero(testData)
XBiClassTest = biClassTest[:, 2:biClassTest.shape[1]]
yBiClassTest = biClassTest[:, 0]

In [5]:
XTrain = XTrain.astype('float')/255
XTest = XTest.astype('float')/255

In [6]:
print(XTrain.shape)
XCCTrain = np.array([np.reshape(XTrain[1][0:138714], (379, 366))]*XTrain.shape[0])
for i in range(1,XTrain.shape[0]):
    XCCTrain[i]=(np.reshape(XTrain[i][0:138714], (379, 366)))
print(XCCTrain.shape)


(486, 275942)
(486, 379, 366)


In [7]:
XCCTrain = XTrain[:,0:138714]
print(XCCTrain.shape)
XCCTrain = XCCTrain.reshape(XTrain.shape[0], 379, 366, 1)
print(XCCTrain.shape)

(486, 138714)
(486, 379, 366, 1)


# Creating the CNN using the CC View

In [8]:
annImageModel = tf.keras.models.Sequential()
#first argument = number of output in the layer 
#second argument = the filter/kernel size (tuple)
#input_shape = (batch_size, image size pixel, image size pixel, depth)
#in this case, 32 x 32 RGB picture with depth of 3
annImageModel.add(tf.keras.layers.Conv2D(64, (2, 2), activation='tanh', input_shape=(379, 366, 1), kernel_regularizer=tf.keras.regularizers.l2(0.01), bias_regularizer = tf.keras.regularizers.l2(0.01), activity_regularizer = tf.keras.regularizers.l2(0.01)))
#padding = 'valid' -> adds no padding
annImageModel.add(tf.keras.layers.MaxPooling2D((2, 2), padding = 'valid'))
annImageModel.add(tf.keras.layers.Conv2D(128, (2, 2), activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.01), bias_regularizer = tf.keras.regularizers.l2(0.01), activity_regularizer = tf.keras.regularizers.l2(0.01)))
annImageModel.add(tf.keras.layers.MaxPooling2D((2, 2), padding = 'valid'))
annImageModel.add(tf.keras.layers.Conv2D(128, (2, 2), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), bias_regularizer = tf.keras.regularizers.l2(0.01), activity_regularizer = tf.keras.regularizers.l2(0.01)))
#changing the model from 3D to 1D (where the input of dense has to be 1D)
annImageModel.add(tf.keras.layers.Flatten())
annImageModel.add(tf.keras.layers.Dense(64, activation = 'tanh'))
#annImageModel.add(tf.keras.layers.Dense(28, activation = 'tanh'))
#annImageModel.add(tf.keras.layers.Dense(8, activation = 'tanh'))
annImageModel.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))
optimizer = tf.optimizers.Adam(lr = 0.000001)
annImageModel.compile(optimizer, loss = 'binary_crossentropy', metrics =['accuracy'])
annImageModel.summary()
#train the model
annImageModel.fit(XCCTrain, yMoBTrain, batch_size = 10, epochs = 100)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 378, 365, 64)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 189, 182, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 188, 181, 128)     32896     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 94, 90, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 93, 89, 128)       65664     
_________________________________________________________________
flatten (Flatten)            (None, 1059456)           0         
_________________________________________________________________
dense (Dense)                (None, 64)                6

486/486 [==============================] - 86s 178ms/sample - loss: 86966.1880 - accuracy: 0.8354
Epoch 62/100
486/486 [==============================] - 88s 181ms/sample - loss: 86791.5347 - accuracy: 0.8519
Epoch 63/100
486/486 [==============================] - 90s 186ms/sample - loss: 86613.8889 - accuracy: 0.8230
Epoch 64/100
486/486 [==============================] - 93s 191ms/sample - loss: 86432.9173 - accuracy: 0.8498
Epoch 65/100
486/486 [==============================] - 92s 188ms/sample - loss: 86247.7974 - accuracy: 0.8704
Epoch 66/100
486/486 [==============================] - 90s 185ms/sample - loss: 86058.3551 - accuracy: 0.8992
Epoch 67/100
486/486 [==============================] - 91s 188ms/sample - loss: 85864.3005 - accuracy: 0.8930
Epoch 68/100
486/486 [==============================] - 92s 190ms/sample - loss: 85665.4818 - accuracy: 0.8354
Epoch 69/100
486/486 [==============================] - 90s 185ms/sample - loss: 85462.3681 - accuracy: 0.8909
Epoch 70/100
4

# Testing the Model

In [10]:

XCCTest = XTest[:,0:138714]
#print(XCCTest.shape)
XCCTest = XCCTest.reshape(XTest.shape[0], 379, 366, 1)
#print(XCCTrain.shape)
annImageModel.evaluate(XCCTest, yMoBTest)

146/146 [==============================] - 7s 49ms/sample - loss: 78978.4403 - accuracy: 0.6986


[78978.44028253424, 0.69863015]

# CNN Using the MLO View

In [13]:
XMLOTrain = XTrain[:,138714:]
print(XMLOTrain.shape)
XMLOTrain = XMLOTrain.reshape(XTrain.shape[0], 377, 364, 1)
print(XMLOTrain.shape)

(486, 137228)
(486, 377, 364, 1)


In [16]:
annImageModel = tf.keras.models.Sequential()
#first argument = number of output in the layer 
#second argument = the filter/kernel size (tuple)
#input_shape = (batch_size, image size pixel, image size pixel, depth)
#in this case, 32 x 32 RGB picture with depth of 3
annImageModel.add(tf.keras.layers.Conv2D(64, (2, 2), activation='tanh', input_shape=(377, 364, 1), kernel_regularizer=tf.keras.regularizers.l2(0.01), bias_regularizer = tf.keras.regularizers.l2(0.01), activity_regularizer = tf.keras.regularizers.l2(0.01)))
#padding = 'valid' -> adds no padding
annImageModel.add(tf.keras.layers.MaxPooling2D((2, 2), padding = 'valid'))
annImageModel.add(tf.keras.layers.Conv2D(128, (2, 2), activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.01), bias_regularizer = tf.keras.regularizers.l2(0.01), activity_regularizer = tf.keras.regularizers.l2(0.01)))
annImageModel.add(tf.keras.layers.MaxPooling2D((2, 2), padding = 'valid'))
annImageModel.add(tf.keras.layers.Conv2D(128, (2, 2), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), bias_regularizer = tf.keras.regularizers.l2(0.01), activity_regularizer = tf.keras.regularizers.l2(0.01)))
#changing the model from 3D to 1D (where the input of dense has to be 1D)
annImageModel.add(tf.keras.layers.Flatten())
annImageModel.add(tf.keras.layers.Dense(64, activation = 'tanh'))
#annImageModel.add(tf.keras.layers.Dense(28, activation = 'tanh'))
#annImageModel.add(tf.keras.layers.Dense(8, activation = 'tanh'))
annImageModel.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))
optimizer = tf.optimizers.Adam(lr = 0.000001)
annImageModel.compile(optimizer, loss = 'binary_crossentropy', metrics =['accuracy'])
annImageModel.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 376, 363, 64)      320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 188, 181, 64)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 187, 180, 128)     32896     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 93, 90, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 92, 89, 128)       65664     
_________________________________________________________________
flatten_2 (Flatten)          (None, 1048064)           0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)               

In [17]:
annImageModel.fit(XMLOTrain, yMoBTrain, batch_size = 10, epochs = 100)

Train on 486 samples
Epoch 1/100
486/486 [==============================] - 84s 172ms/sample - loss: 96423.8741 - accuracy: 0.4465
Epoch 2/100
486/486 [==============================] - 88s 181ms/sample - loss: 95983.6695 - accuracy: 0.4897
Epoch 3/100
486/486 [==============================] - 85s 174ms/sample - loss: 95561.3632 - accuracy: 0.5267
Epoch 4/100
486/486 [==============================] - 88s 180ms/sample - loss: 95156.2622 - accuracy: 0.5103
Epoch 5/100
486/486 [==============================] - 87s 179ms/sample - loss: 94766.9660 - accuracy: 0.5556
Epoch 6/100
486/486 [==============================] - 88s 180ms/sample - loss: 94392.9989 - accuracy: 0.5309
Epoch 7/100
486/486 [==============================] - 86s 176ms/sample - loss: 94033.0540 - accuracy: 0.5021
Epoch 8/100
486/486 [==============================] - 87s 179ms/sample - loss: 93686.1370 - accuracy: 0.5082
Epoch 9/100
486/486 [==============================] - 88s 181ms/sample - loss: 93350.8831 - accura

In [18]:

XMLOTest = XTest[:,138714:]
#print(XCCTest.shape)
XMLOTest = XMLOTest.reshape(XTest.shape[0], 377, 364, 1)
#print(XCCTrain.shape)
annImageModel.evaluate(XMLOTest, yMoBTest)

146/146 [==============================] - 6s 43ms/sample - loss: 69321.6714 - accuracy: 0.6370


[69321.67144691781, 0.6369863]